# 模块与包

## 构建一个模块的层级包

包结构：

```
graphics/
    __init__.py
    primitive/
        __init__.py
        line.py
        fill.py
        text.py
    formats/
        __init__.py
        png.py
        jpg.py
```

__init__.py文件用来自动加载子模块

则对应的使用方法：

```python
import graphics.primitive.line
from graphics.primitive import line
import graphics.formats.jpg as jpg
```

## 控制模块被全部导入的内容

当使用`from module import *`语句时，可以定义一个变量__all__来明确地列出需要导出地内容

In [1]:
# 一个py文件内

def spam():
    pass

def grok():
    pass

# 只能导出spam和grok
__all__ = ['spam', 'gork']

## 使用相对路径名导入包中子模块

文件结构：

```
mypackage/
    __init__.py
    A/
        __init__.py
        spam.py
        grok.py
    B/
        __init__.py
        bar.py
```

如果模块 mypackage.A.spam 要导入同目录下的模块 grok，它应该包括的 import语句如下：

```python
from . import grok
```

如果模块 mypackage.A.spam 要导入不同目录下的模块 B.bar，它应该使用的import 语句如下：

```python
from ..B import bar
```

## 将模块分割成多个文件

考虑下下面简单的模块：

```python
class A:
    def spam(self):
        print('A.spam')
class B(A):
    def bar(self):
        print('B.bar')
```

分割成：

```
mymodule/
    __init__.py
    a.py
    b.py
```

其中a.py：

```python
class A:
    def spam(self):
        print('A.spam')
```

其中b.py：

```python
class B(A):
    def bar(self):
        print('B.bar')
```

最后，在 __init__.py 中，将 2 个文件粘合在一起：

```python
from .a import A
from .b import B
```

所产生的包MyModule将作为一个单一的逻辑模块：

```python
import mymodule
a = mymodule.A()
b = mymodule.B()
```

## 利用命名空间导入目录分散的代码

两个包：

```
foo-package/
    spam/
        blah.py
bar-package/
    spam/
        grok.py
```

合并导入

```python
import sys
sys.path.extend(['foo-package', 'bar-package'])
import spam.blah
import spam.grok
```


## 重新加载模块

使用imp.read()来重新加载先前加载的模块：

```python
import spam # 修改过的py文件，重新加载
import imp
imp.reload(spam)
```

## 运行目录或压缩文件

文件结构：

```
myapplication/
    spam.py
    bar.py
    grok.py
    __main__.py
```

目录下直接：

```bash
python3 myapplication
```

打包成zip文件

```
bash % ls
spam.py bar.py grok.py __main__.py
bash % zip -r myapp.zip *.py
bash % python3 myapp.zip
... output from __main__.py ...
```

## 读取位于包中的数据文件

文件组织:

```
mypackage/
    __init__.py
    somedata.dat
    spam.py
```

现在假设 spam.py 文件需要读取 somedata.dat 文件中的内容。可以用以下代码来完成：
```python
import pkgutil
data = pkgutil.get_data(__package__, 'somedata.dat')
```

由此产生的变量是包含该文件的原始内容的字节字符串。

## 将文件夹加入到 sys.path

1. 使用 PYTHONPATH环境变量来添加
2. 创建一个.pth 文件，将目录列举出来

## 通过字符串名导入模块

In [2]:
import importlib

math = importlib.import_module('math')
print(math.sin(2))

0.9092974268256817


## 通过钩子远程加载模块

略

## 导入模块的同时修改模块

In [3]:
import importlib
import sys
from collections import defaultdict

_post_import_hooks = defaultdict(list)

class PostImportFinder:
    def __init__(self):
        self._skip = set()

    def find_module(self, name, path=None):
        if name in self._skip:
            return None
        self._skip.add(name)
        return PostImportLoader(self)

class PostImportLoader:
    def __init__(self, finder):
        self._finder = finder

    def load_module(self, name):
        importlib.import_module(name)
        module = sys.modules[name]
        for func in _post_import_hooks[name]:
            func(module)
        self._finder._skip.remove(name)
        return module

def when_imported(name):
    def decorate(func):
        if name in sys.modules:
            func(sys.modules[name])
        else:
            _post_import_hooks[name].append(func)
        return func
    return decorate

sys.meta_path.insert(0, PostImportFinder())

In [4]:
@when_imported('threading')
def warn_thread(mod):
    print('Threads? Are you crazy?')

import threading

Threads? Are you crazy?


## 安装私有的包

pip install --user packagename

## 创建新的 Python 环境

使用pyvenv命令

## 分发包

典型的项目结构：

```
projectname/
    README.txt
    Doc/
        documentation.txt
    projectname/
        __init__.py
        foo.py
        bar.py
        utils/
            __init__.py
            spam.py
            grok.py
    examples/
        helloworld.py
```

要让你的包可以发布出去，首先你要编写一个 setup.py ，类似下面这样：

```python
from distutils.core import setup

setup(name='projectname',
    version='1.0',
    author='Your Name',
    author_email='you@youraddress.com',
    url='http://www.you.com/projectname',
    packages=['projectname', 'projectname.utils'],
)
```

下一步，就是创建一个 MANIFEST.in 文件，列出所有在你的包中需要包含进来的非源码文件：

```
# MANIFEST.in
include *.txt
recursive-include examples *
recursive-include Doc *
```

确保 setup.py 和 MANIFEST.in 文件放在包的最顶级目录中,就可以像下面这样执行命令来创建一个源码分发包了：

```
python3 setup.py sdist
```

它会创建一个文件比如”projectname-1.0.zip”或“projectname-1.0.tar.gz”, 具体依赖于你的系统平台。如果一切正常，这个文件就可以发送给别人使用或者上传至Python Package Index.